# BoolQ

## Creating vector store

In [1]:
!python ../scripts/create_vector_store.py \
  --dataset "boolq" \
  --emb_model "BAAI/bge-small-en-v1.5" \
  --cs 150 \
  --co 20 \
  --bs_emb 1024 \
  --output_dir "../vector_stores/boolq/base_"

Load model embedding
Using device: cuda
100%|████████████████████████████████████| 12697/12697 [00:13<00:00, 971.19it/s]
Total number of passages: 16519
Removing duplicate passages
Total number of passages created: 13091
Creating vector store
Load model embedding : BAAI/bge-small-en-v1.5
Using device: cuda
Generando embeddings: 100%|█████████████████████| 13/13 [00:11<00:00,  1.16it/s]
✅ Índice FAISS creado exitosamente.
💾 Vector store saved in ../vector_stores/boolq/base_vs_boolq_150_20


## Train embedding

In [2]:
!python  ../scripts/train_embedding.py \
  --name_dataset "boolq" \
  --model_name "BAAI/bge-small-en-v1.5" \
  --new_model_name "bge-small-boolq" \
  --epochs 10 \
  --batch_size 128 \
  --output_dir "../models/boolq/embedding/"

wandb: Currently logged in as: dinho15971 (dinho15971-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Starting main process...
Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Datasets loaded and prepared.
Creating evaluator...
Evaluator created.
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /local1/ronaldinho/projects/test_sbbd/notebooks2/wandb/run-20250503_181644-c0dq8gdx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bge-small-boolq_10e_128bs
wandb: ⭐️ View project at https://wandb.ai/dinho15971-unicamp/SBBD_embeddings
wandb: 🚀 View run at https://wandb.ai/dinho15971-unicamp/SBBD_embeddings/runs/c0dq8gdx
Loading model: BAAI/bge-small-en-v1.5
Model and loss function initialized.
Configuring training arguments...
Training arguments configured.
Starting training process...
{'loss': 0.632, 'grad_norm': 3.0354115962982178

## Evaluate embedding

In [3]:
!python  ../scripts/evaluate_embedding.py \
  --name_dataset "boolq" \
  --output_dir "../results/boolq/" \
  --models_dir "../models/boolq/embedding/"

Cargando modelos...
Models loaded
Loading datasets...
Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Datasets loaded and prepared.
Loaded dataset
Creating evaluator...
Evaluator created.
Evaluating models
Save results...


## Creating New Vector Store

In [4]:
!python ../scripts/create_vector_store.py \
  --dataset "boolq" \
  --emb_model "../models/boolq/embedding/bge-small-boolq_10e_128bs" \
  --cs 150 \
  --co 20 \
  --bs_emb 1024 \
  --output_dir "../vector_stores/boolq/ft_"

Load model embedding
Using device: cuda
100%|████████████████████████████████████| 12697/12697 [00:13<00:00, 947.80it/s]
Total number of passages: 16519
Removing duplicate passages
Total number of passages created: 13091
Creating vector store
Load model embedding : ../models/boolq/embedding/bge-small-boolq_10e_128bs
Using device: cuda
Generando embeddings: 100%|█████████████████████| 13/13 [00:11<00:00,  1.13it/s]
✅ Índice FAISS creado exitosamente.
💾 Vector store saved in ../vector_stores/boolq/ft_vs_boolq_150_20


## Train Phi-2

In [5]:
import sys
from utils.data_for_train_phi import get_dataset_for_train_phi
from vector_stores.faiss import VectorStoreFaiss
vector_store = VectorStoreFaiss.load_local("../vector_stores/boolq/ft_vs_boolq_150_20")
train_ds, test_ds = get_dataset_for_train_phi('boolq', True, vector_store,4, 8)

Load model embedding : ../models/boolq/embedding/bge-small-boolq_10e_128bs
Using device: cuda
💾 Vector store loaded from../vector_stores/boolq/ft_vs_boolq_150_20
Creating dataset for boolq
Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.


🔍 Buscando: 100%|████████████████████████| 943/943 [00:11<00:00, 80.77it/s]


Map:   0%|          | 0/7541 [00:00<?, ? examples/s]

🔍 Buscando: 100%|████████████████████████| 236/236 [00:02<00:00, 85.32it/s]


Flattening the indices:   0%|          | 0/1886 [00:00<?, ? examples/s]

Map:   0%|          | 0/1886 [00:00<?, ? examples/s]

In [6]:
print(train_ds[0]['text'])

Instruct: You are given a context and a question. Answer only "yes" or "no" based solely on the information contained in the context. If the context does not provide enough information to answer confidently, answer based on the most likely interpretation from the given text.

Context:

Document 0:. Its primary inlet is the Niagara River from Lake Erie. The last in the Great Lakes chain, Lake Ontario serves as the outlet to the Atlantic Ocean via the Saint Lawrence River.
Document 1:The Great Lakes Waterway is a system of natural channels and canals which enable navigation between the North American Great Lakes. Though all of the lakes are naturally connected as a chain, water travel between the lakes was impeded for centuries by obstacles such as Niagara Falls and the rapids of the St. Marys River.
Document 2:The Welland Canal is a ship canal in Ontario, Canada, connecting Lake Ontario and Lake Erie. It forms a key section of the St. Lawrence Seaway. Traversing the Niagara Peninsula fr

In [ ]:
!python  ../scripts/ft_phi.py \
  --new_model_name "phi_2_rag_k1_boolq_2e_10bs" \
  --num_epochs 2 \
  --batch_size 10 \
  --dataset_name "boolq" \
  --include_docs \
  --top_k 1 \
  --save_path "../models/boolq/adapters/" \
  --vector_store_path "../vector_stores/boolq/ft_vs_boolq_150_20"

wandb: Currently logged in as: dinho15971 (dinho15971-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /local1/ronaldinho/projects/test_sbbd/notebooks2/wandb/run-20250503_200104-8y4x7p82
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run phi_2_rag_k1_boolq_2e_10bs
wandb: ⭐️ View project at https://wandb.ai/dinho15971-unicamp/SBBD_phi-2-adapters
wandb: 🚀 View run at https://wandb.ai/dinho15971-unicamp/SBBD_phi-2-adapters/runs/8y4x7p82
Usando dispositivo: cuda
Load model embedding : ../models/boolq/embedding/bge-small-boolq_10e_128bs
Using device: cuda
💾 Vector store loaded from../vector_stores/boolq/ft_vs_boolq_150_20
Using k = 1 passages
Creating dataset for boolq
Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Map: 100%|███████████████████████████| 1886/1886 [07:11<00:00,  4.37

## Inference

In [4]:
!python ../scripts/inference_rag.py \
  --lora_adapter_path "../models/boolq/adapters/best_phi_2_rag_k1_boolq_2e_10bs" \
  --max_new_tokens 10 \
  --vector_store_path "../vector_stores/boolq/ft_vs_boolq_150_20" \
  --dataset_name "boolq" \
  --output_csv_path "../results/boolq/full_ft_k10_boolq.csv" \
  --bs_emb 50 \
  --bs_gen 8 \
  --top_k 10 \
  --use_rag

Using device: cuda
Loading base model from LoRA adapter: ../models/boolq/adapters/best_phi_2_rag_k1_boolq_2e_10bs
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00, 18.77it/s]
Load model embedding : ../models/boolq/embedding/bge-small-boolq_10e_128bs
Using device: cuda
💾 Vector store loaded from../vector_stores/boolq/ft_vs_boolq_150_20
Device set to use cuda:0
Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Dataset({
    features: ['question', 'answer', 'passage'],
    num_rows: 3270
})
Generating prompts: 100%|████████████████| 3270/3270 [00:00<00:00, 16677.44it/s]
Example prompt
Instruct: You are given a context and a question. Answer only "yes" or "no" based solely on the information contained in the context. If the context does not provide enough information to answer confidently, answer based on the most likely interpretation from the given text.

Context:

Document 1:.4×10 U.S. ga

## Evaluation

In [2]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('boolq', '../results/boolq/full_ft_k10_boolq.csv')

Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Results saved in: ../results/boolq/full_ft_k10_boolq.csv
